<a href="https://colab.research.google.com/github/Shreyansh19-o/Gen_AI_Tools_Calling/blob/main/tool_calling_in_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import os
os.environ["GOOGLE_API_KEY"] = " "

In [153]:
!pip install -q langchain-openai langchain-core requests langchain-google-genai

In [154]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [155]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [156]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [157]:
multiply.name

'multiply'

In [158]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [159]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [160]:
# tool binding

In [161]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)

In [162]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--0e40bce4-2287-4471-8dae-ec5f9be47cb0-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [163]:
llm_with_tools = llm.bind_tools([multiply])

In [164]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content='I am doing well, thank you for asking. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--cb607939-84a0-46b1-9eb7-b9f00b2ce714-0', usage_metadata={'input_tokens': 24, 'output_tokens': 18, 'total_tokens': 42, 'input_token_details': {'cache_read': 0}})

In [165]:
query = HumanMessage('can you multiply 3 with 1000')

In [166]:
messages = [query]

In [167]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [168]:
result = llm_with_tools.invoke(messages)

In [169]:
messages.append(result)

In [170]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6be56bba-80a7-4fbf-acc8-ee62cd199adf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': 'aaf5b406-8734-452f-914c-ede87abbcfa2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})]

In [171]:
tool_result = multiply.invoke(result.tool_calls[0])

In [172]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='aaf5b406-8734-452f-914c-ede87abbcfa2')

In [173]:
messages.append(tool_result)

In [174]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6be56bba-80a7-4fbf-acc8-ee62cd199adf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': 'aaf5b406-8734-452f-914c-ede87abbcfa2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='aaf5b406-8734-452f-914c-ede87abbcfa2')]

In [175]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is 3000.'

In [176]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [177]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [178]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1754870402,
 'time_last_update_utc': 'Mon, 11 Aug 2025 00:00:02 +0000',
 'time_next_update_unix': 1754956802,
 'time_next_update_utc': 'Tue, 12 Aug 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.625}

In [179]:
convert.invoke({'base_currency_value':10, 'conversion_rate':87.625})

876.25

In [180]:
# tool binding
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [181]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [182]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [183]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [184]:
ai_message = llm_with_tools.invoke(messages)

In [185]:
messages.append(ai_message)

In [186]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': '551ba7ef-82c8-4d02-acb3-cdf27f899771',
  'type': 'tool_call'}]

In [187]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [188]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6cc1b528-ef3f-4058-8cca-530b029cdf4d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': '551ba7ef-82c8-4d02-acb3-cdf27f899771', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 13, 'total_tokens': 105, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms",

In [189]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01141.'